## First, I'll load the required data and setup keras settings

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
# use non standard flow_from_directory
from utils.image_preprocessing_ver2 import ImageDataGenerator
# it outputs y_batch that contains embeddings

from utils.mobilenet import get_mobilenet
import keras
from keras.applications.mobilenet import MobileNet
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.models import Model
from keras import optimizers
from keras import regularizers
from keras import backend as K
from keras.layers import Lambda, concatenate, Activation
from keras.losses import categorical_crossentropy as logloss
from keras.metrics import categorical_accuracy, top_k_categorical_accuracy
import matplotlib.pyplot as plt
from keras.layers import Activation, GlobalAveragePooling2D, Dropout, Dense, Input
%matplotlib inline

Using TensorFlow backend.


In [3]:
logit_dir = 'data/face_emb_logits/'

In [4]:
train_logits = np.load(logit_dir + 'train_set.npy')[()]
val_logits = np.load(logit_dir + 'test_set.npy')[()]


In [5]:
data_dir = "/home/prudhvi/Desktop/Datasets/lfw/lfw_mtcnn_160_split/"

In [6]:
def fixed_image_stand(image):
    return (tf.cast(image, tf.float32) - 127.5)/128

In [7]:
train_generator = ImageDataGenerator(
    data_format='channels_last',
    preprocessing_function = fixed_image_stand
)

test_generator = ImageDataGenerator(
    data_format='channels_last',
    preprocessing_function = fixed_image_stand
)

# note: i'm also passing dicts of logits
train_generator = train_generator.flow_from_directory(
    data_dir + 'train/', train_logits,
    target_size=(160, 160),
    batch_size=64,
    class_mode = 'embedding',
)

val_generator = test_generator.flow_from_directory(
    data_dir + 'test', val_logits,
    target_size=(160, 160),
    batch_size=64,
    class_mode = 'embedding',
)

Found 12020 images belonging to 1 classes.
Found 1213 images belonging to 1 classes.


## Create your Mobilenet model

In [8]:
def get_mobilenet(input_size = 160, alpha = 1 ):
    
    input_shape = (input_size, input_size, 3)
    base_model = MobileNet(
        include_top=False, weights='imagenet', 
        input_shape=input_shape, alpha=alpha
    )
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.25)(x)
    logits = Dense(512, kernel_regularizer = regularizers.l2(0.001))(x)
    l2_norm = Lambda(lambda  x: K.l2_normalize(x,axis=1))(logits)
    model = Model(base_model.input, l2_norm)
    
    for layer in model.layers[:-2]:
        layer.trainable = False
    return model

model = get_mobilenet()
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 160, 160, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 162, 162, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 80, 80, 32)        864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 80, 80, 32)        128       
_________________________________________________________________
conv1_relu (Activation)      (None, 80, 80, 32)        0         
_________________________________________________________________
conv_pad_1 (ZeroPadding2D)   (None, 82, 82, 32)        0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 80, 80, 32)        288       
__________

In [9]:
def embedding_loss(y_true, y_pred):    
    
    return K.mean(K.sum(K.square(y_true - y_pred), axis = 1))

In [10]:

model.compile(
    optimizer=optimizers.SGD(lr=1e-1, momentum=0.9, nesterov=True), #SGD has the best generalization capability
#     optimizer = optimizers.RMSprop(lr=1e-5),
    loss=embedding_loss, 
    metrics=[embedding_loss]
)

In [ ]:
model.fit_generator(
    train_generator, 
    steps_per_epoch=400, epochs=30, verbose=1,
    callbacks=[
        ReduceLROnPlateau(monitor='val_embedding_loss', factor=0.1, patience=2, epsilon=0.007)
    ],
    validation_data=val_generator, validation_steps=80, workers=4
)

/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:999: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


Epoch 1/30
400/400 [==============================] - 145s 362ms/step - loss: 1.7536 - embedding_loss: 1.3935 - val_loss: 1.6087 - val_embedding_loss: 1.4504
Epoch 2/30
400/400 [==============================] - 84s 209ms/step - loss: 1.2481 - embedding_loss: 1.1481 - val_loss: 1.5195 - val_embedding_loss: 1.4477
Epoch 3/30
400/400 [==============================] - 42s 105ms/step - loss: 1.1864 - embedding_loss: 1.1179 - val_loss: 1.4633 - val_embedding_loss: 1.3959
Epoch 4/30
400/400 [==============================] - 43s 107ms/step - loss: 1.1857 - embedding_loss: 1.1181 - val_loss: 1.4390 - val_embedding_loss: 1.3712
Epoch 5/30
400/400 [==============================] - 42s 106ms/step - loss: 1.1862 - embedding_loss: 1.1185 - val_loss: 1.5355 - val_embedding_loss: 1.4679
Epoch 6/30
400/400 [==============================] - 43s 107ms/step - loss: 1.1875 - embedding_loss: 1.1198 - val_loss: 1.5132 - val_embedding_loss: 1.4454
Epoch 7/30
397/400 [============================>.] - ETA

### Loss/epoch plots

In [ ]:
plt.plot(model.history.history['embedding_loss'], label='train');
plt.plot(model.history.history['val_embedding_loss'], label='val');
plt.legend();
plt.xlabel('epoch');
plt.ylabel('emb_loss');

### Save your model to disk

In [57]:
model.save("mobnet_distill.h5")